Connect to the SQL database.

In [1]:
import duckdb
import pandas as pd
import ipywidgets

%reload_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///vcf_dfs.db

ModuleNotFoundError: No module named 'duckdb'

In [ ]:

%%sql
SET memory_limit = '10GB';
SET enable_progress_bar = true;

,Success


Define the locations of the references and samples, and quality/depth cutoffs.

In [ ]:
ref_parent_file = "../../data/parquets/w1118.parquet"
alt_parent_file = "../../data/parquets/oregonr.parquet"
sample_file_glob = "../../data/parquets/WT-G0-*.parquet"
ref_parent_name = "w1118"
alt_parent_name = "oregonr"

ref_qual_cutoff = 200
ref_depth_cutoff = 30

ref_out_file = "../../data/parquets/reference.parquet"
sample_out_file = "../../data/parquets/progeny.parquet"

Import the reference files as a table. 

In [ ]:
%%sql
CREATE OR REPLACE VIEW parents AS
SELECT sample, chromosome, CAST(position AS INTEGER) AS int_pos, quality, genotype, depth, allele_depth, reference, variant, (CASE WHEN variant='.' THEN reference ELSE variant END) AS mod_variant FROM read_parquet(['{{ref_parent_file}}', '{{alt_parent_file}}']);


RuntimeError: (duckdb.duckdb.BinderException) Binder Error: GROUP BY clause cannot contain aggregates!
LINE 2: GROUP BY ANY_VALUE(sample), chromosome, int_pos;
                 ^
[SQL: SELECT * FROM parents
GROUP BY ANY_VALUE(sample), chromosome, int_pos;]
(Background on this error at: https://sqlalche.me/e/20/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


Isolate sites where there are 2 variants (meaning one parent is different from the other).

In [ ]:
%%sql
CREATE OR REPLACE VIEW check_unique_variants AS
SELECT chromosome, int_pos, COUNT(variant) AS n_variants FROM parents
    GROUP BY chromosome, int_pos
    HAVING n_variants >= 2;

,Success


Create new reference from the reference parent.

In [ ]:
%%sql
CREATE OR REPLACE VIEW temp_ref AS
SELECT
    sample,
    parents.chromosome,
    parents.int_pos,
    reference,
    mod_variant,
    quality,
    genotype,
    depth,
    allele_depth
    FROM parents
    INNER JOIN check_unique_variants ON parents.chromosome = check_unique_variants.chromosome AND parents.int_pos = check_unique_variants.int_pos
    WHERE genotype!='0/1'
        AND quality > '{{ref_qual_cutoff}}'
        AND LENGTH(mod_variant) <= 1
        AND LENGTH(reference) <= 1
        AND depth > '{{ref_depth_cutoff}}';

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,sample,chromosome,int_pos,reference,mod_variant,quality,genotype,depth,allele_depth
0,w1118_combined,chr2L,13247,C,T,221.998993,1/1,16,"0,16"
1,w1118_combined,chr2L,13638,G,A,222.000000,1/1,13,"0,13"
2,oregonr_combined,chr2L,13638,G,A,221.998993,1/1,31,"0,31"
3,w1118_combined,chr2L,20008,A,G,221.998993,1/1,20,"0,20"
4,w1118_combined,chr2L,28813,A,G,221.998993,1/1,17,"0,17"
...,...,...,...,...,...,...,...,...,...
276434,w1118_combined,chrY,975741,A,T,221.998993,1/1,175,"7,168"
276435,oregonr_combined,chrY,975741,A,T,221.998993,1/1,275,"26,249"
276436,w1118_combined,chrY,975783,T,A,221.998993,1/1,171,"3,168"
276437,oregonr_combined,chrY,975783,T,A,221.998993,1/1,263,"8,255"


In [ ]:
%%sql

CREATE OR REPLACE VIEW ref_parent AS
SELECT sample, chromosome, int_pos, mod_variant AS ref_allele FROM temp_ref
WHERE sample='{{ref_parent_name}}';

CREATE OR REPLACE VIEW alt_parent AS
SELECT sample, chromosome, int_pos, mod_variant AS alt_allele FROM temp_ref
WHERE sample='{{alt_parent_name}}';

CREATE OR REPLACE TABLE ref AS
SELECT * FROM ref_parent
FULL JOIN alt_parent ON (ref_parent.chromosome = alt_parent.chromosome AND ref_parent.int_pos = alt_parent.int_pos)
ORDER BY ref_parent.chromosome, ref_parent.int_pos;

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,sample,chromosome,int_pos,ref_allele,sample_1,chromosome_1,int_pos_1,alt_allele
0,w1118_combined,chr2L,13247.0,T,None,None,NaN,None
1,w1118_combined,chr2L,13638.0,A,oregonr_combined,chr2L,13638.0,A
2,w1118_combined,chr2L,20008.0,G,None,None,NaN,None
3,w1118_combined,chr2L,28813.0,G,oregonr_combined,chr2L,28813.0,G
4,w1118_combined,chr2L,36330.0,T,oregonr_combined,chr2L,36330.0,T
...,...,...,...,...,...,...,...,...
175393,None,None,NaN,None,oregonr_combined,chr2L,286991.0,A
175394,None,None,NaN,None,oregonr_combined,chr2L,65405.0,T
175395,None,None,NaN,None,oregonr_combined,chr2R,16119415.0,G
175396,None,None,NaN,None,oregonr_combined,chr2R,16126653.0,A


In [ ]:

%%sql

CREATE OR REPLACE VIEW temp_vcfs AS
SELECT * FROM read_parquet('{{sample_file_glob}}');

CREATE OR REPLACE VIEW vcfs AS
SELECT
    sample,
    chromosome,
    CAST(position AS INTEGER) AS int_pos,
    reference,
    variant,
    quality,
    genotype,
    depth,
    allele_depth,
    (CASE WHEN temp_vcfs.variant='.' THEN temp_vcfs.reference ELSE temp_vcfs.variant END) AS new_variant
    FROM temp_vcfs;

CREATE OR REPLACE VIEW samples_rearranged AS
SELECT *, (CASE WHEN ref_allele=new_variant THEN '.' ELSE new_variant END) AS var_adjusted FROM vcfs
    INNER JOIN ref ON vcfs.chromosome = ref.chromosome AND vcfs.int_pos = ref.int_pos
    WHERE new_variant=ref_allele OR new_variant=alt_allele;

CREATE OR REPLACE TABLE final_samples AS
SELECT 
    string_split(sample, '-')[1] AS condition,
    string_split(sample, '-')[2] AS sample_type,
    string_split(sample, '-')[3] AS sample_num,
    reference,
    variant,
    chromosome,
    int_pos AS position,
    string_split(allele_depth, ',')[1] AS ref_reads,
    string_split(allele_depth, ',')[2] AS variant_reads,
    quality AS QUAL,
    genotype AS GT,
    depth AS DP
    FROM samples_rearranged
ORDER BY sample_num, chromosome, position;

CHECKPOINT;

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [ ]:
%%sql

COPY (SELECT * FROM final_samples)
TO '{{ref_out_file}}'
(FORMAT 'parquet');

COPY (SELECT chromosome, int_pos AS position, ref_allele AS reference, alt_allele AS variant FROM ref)
TO '{{sample_out_file}}'
(FORMAT 'parquet');


,condition,sample_type,sample_num,reference,variant,chromosome,position,ref_reads,variant_reads,QUAL,GT,DP
0,WT,G0,001,G,A,chr2L,1023412,0,87,221.998993,1/1,87
1,WT,G0,001,C,T,chr2L,1150444,0,88,221.998993,1/1,88
2,WT,G0,001,A,C,chr2L,1617652,0,28,221.998993,1/1,28
3,WT,G0,001,G,A,chr2L,1712703,0,70,221.998993,1/1,70
4,WT,G0,001,G,C,chr2L,1754752,0,75,221.998993,1/1,75
...,...,...,...,...,...,...,...,...,...,...,...,...
1448,WT,G0,016,C,A,chr3R,30001926,0,97,221.998993,1/1,97
1449,WT,G0,016,T,G,chr3R,31655172,26,20,192.009003,0/1,46
1450,WT,G0,016,T,G,chrX,10005635,46,59,225.009003,0/1,105
1451,WT,G0,016,A,C,chrX,15677336,0,152,221.998993,1/1,152
